In [3]:
# from sophy_util import chat, company_qna_chat
from langchain.prompts import ChatPromptTemplate
from sophy_util import chat, company_qna_chat, exchanges_pretty
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from graph_util import display_graph
from typing import Union, Literal
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, Graph, START, END
from langgraph.graph import StateGraph
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_community.vectorstores import FAISS

In [4]:
ss_agent_key = ""
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=ss_agent_key
)
with open('res\company.md',"r") as f:
    markdown_document = f.read()

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_document)
embeddings = OllamaEmbeddings(model="llama3.2:1b")
vectorstore=FAISS.from_documents(md_header_splits, embeddings)
retriever=vectorstore.as_retriever()

class SophyState(TypedDict):
    exchange: int
    conv_sum: str
    conversation_history: list[AnyMessage]
    latest_exchanges: list[AnyMessage]
    user_input: str
    agent_response: str
    
def company_qa(chat_state:SophyState, retriever=retriever):
    exchange = chat_state.get("exchange",0)
    conv_sum = chat_state.get("conv_sum","")
    conversation_history = chat_state.get("conversation_history",[])
    latest_exchanges = chat_state.get("latest_exchanges",[])
    user_input = chat_state["user_input"]
    print("====> Company QnA <====")
    
    exchange,conversation_history = company_qna_chat(llm,retriever,conversation_history,user_input, exchange)

    return {
        "exchange":exchange,
        "conv_sum":conv_sum,
        "conversation_history":conversation_history,
        "latest_exchanges":latest_exchanges,
        "user_input":user_input
    }

<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Saran\AppData\Local\Temp\ipykernel_24148\1015364681.py:6: SyntaxWarning: invalid escape sequence '\c'
  with open('res\company.md',"r") as f:


In [6]:
user_input = "What is serene solace?"
chat_state = SophyState(exchange=0, conv_sum="", conversation_history=[], latest_exchanges=[], user_input=user_input)


In [14]:
exchange = chat_state.get("exchange",0)
conv_sum = chat_state.get("conv_sum","")
conversation_history = chat_state.get("conversation_history",[])
latest_exchanges = chat_state.get("latest_exchanges",[])
user_input = chat_state["user_input"]
print("====> Company QnA <====")
latest_exchanges = conversation_history[-6 :]
latest_exchanges_pretty = exchanges_pretty(latest_exchanges, True)

template = '''You are an assistant for question-answering tasks.
            Use only the context to answer the question.
            If you don't know the answer, just say that you don't know.
            Use ten sentences maximum and keep the answer concise.
            If Serene Solace is mentioned talk only about the company. Do not provide unnecessary reasoning.

            If you find anything unrelated in the latest conversation ignore them.

            Context: {context}

            latest conversation:
            {latest_exchanges_pretty}
            human:{user_query}
            assistant:'''

prompt = ChatPromptTemplate.from_template(template)
d = {"context": lambda x: retriever,  "latest_exchanges_pretty": latest_exchanges_pretty ,"user_query": user_input}

output_parser=StrOutputParser()
def llm_fake(prompt):
    return prompt

rag_chain = (
 {"context": retriever,  "latest_exchanges_pretty": lambda x: latest_exchanges_pretty ,"user_query":  RunnablePassthrough()}
| prompt
# | llm_fake
# | output_parser
)
exchange += 1


model_response = rag_chain.invoke("What is serene solace?")
model_response

====> Company QnA <====


ChatPromptValue(messages=[HumanMessage(content='You are an assistant for question-answering tasks.\n            Use only the context to answer the question.\n            If you don\'t know the answer, just say that you don\'t know.\n            Use ten sentences maximum and keep the answer concise.\n            If Serene Solace is mentioned talk only about the company. Do not provide unnecessary reasoning.\n\n            If you find anything unrelated in the latest conversation ignore them.\n\n            Context: [Document(id=\'1cbd1b5a-b25c-4650-8306-86b809714dac\', metadata={\'Header 1\': \'Serene Solace Overview\', \'Header 2\': "**Serene Solace\'s Mission: Mental First Aid**"}, page_content="We see ourselves as **mental first aid**, offering a **safe space** where users can talk about their feelings and emotions. If you\'re unsure of what you\'re feeling, whether it\'s **stress, loneliness, anxiety**, or something else, we are here to help.  \\n**What We Mean by First Aid:**\\n- O

In [18]:
x = model_response.to_messages()

In [20]:
for i in x:
    print(i.content)

You are an assistant for question-answering tasks.
            Use only the context to answer the question.
            If you don't know the answer, just say that you don't know.
            Use ten sentences maximum and keep the answer concise.
            If Serene Solace is mentioned talk only about the company. Do not provide unnecessary reasoning.

            If you find anything unrelated in the latest conversation ignore them.

            Context: [Document(id='1cbd1b5a-b25c-4650-8306-86b809714dac', metadata={'Header 1': 'Serene Solace Overview', 'Header 2': "**Serene Solace's Mission: Mental First Aid**"}, page_content="We see ourselves as **mental first aid**, offering a **safe space** where users can talk about their feelings and emotions. If you're unsure of what you're feeling, whether it's **stress, loneliness, anxiety**, or something else, we are here to help.  \n**What We Mean by First Aid:**\n- Our goal is to provide immediate support when you need it most, giving yo

In [ ]:
def company_qna_chat(
    llm,
    retriever,
    conversation_history,
    user_input,
    exchange
):
    reset_exchange = -1

    if user_input.lower() in ["exit", "quit", "stop", "q"]:
        print("Goodbye. Take care.")
        print(end="\n\n\n\n\n")
        # print(exchanges_pretty(conversation_history))

    
    latest_exchanges = conversation_history[-6 :]
    latest_exchanges_pretty = exchanges_pretty(latest_exchanges, True)

    template = '''You are sophy of serene solace company, an customer assistant for question-answering tasks.
                    Use only the context to answer the question.
                    If you don't know the answer, just say that you don't know.
                    Use ten sentences maximum and keep the answer concise.
                    If Serene Solace is mentioned talk only about the company. Do not provide unnecessary reasoning.
        
                    Context: {context}

                    latest conversation:
                    {latest_exchanges_pretty}
                    human:{user_query}
                    assistant:'''
    
    prompt = ChatPromptTemplate.from_template(template)


    output_parser=StrOutputParser()

    rag_chain = (
        {"context": retriever,  "latest_exchanges_pretty":latest_exchanges_pretty ,"user_query": user_input}
        | prompt
        | llm
        | output_parser
    )
    exchange += 1


    model_response = rag_chain.invoke(user_input)

    conversation_history.append(HumanMessage(content=user_input))
    conversation_history.append(AIMessage(content=model_response))
    # Print the response
    print(f"Sophy: {model_response.content}")

    return exchange, conversation_history

In [ ]:
[Document(id='1cbd1b5a-b25c-4650-8306-86b809714dac', metadata={'Header 1': 'Serene Solace Overview', 'Header 2': "**Serene Solace's Mission: Mental First Aid**"}, page_content="We see ourselves as **mental first aid**, offering a **safe space** where users can talk about their feelings and emotions. If you're unsure of what you're feeling, whether it's **stress, loneliness, anxiety**, or something else, we are here to help.  \n**What We Mean by First Aid:**\n- Our goal is to provide immediate support when you need it most, giving you a space to express yourself and be heard.\n- We listen without judgment and help you begin understanding your emotions and challenges, pointing you toward helpful resources.  \n---"), Document(id='ef0a7597-429d-4725-8084-96b2fd334d3c', metadata={'Header 1': 'Serene Solace Overview', 'Header 2': '**How Serene Solace Is Making a Change**'}, page_content='We offer **trained bot** which is equipped to:\n- Provide the **initial emotional support** necessary for people feeling overwhelmed.\n- Help users identify the root of their feelings.\n- Guide individuals to the right resources and help them understand when they might need **professional therapy**.  \n---'), Document(id='c5eb7bc7-46b5-4fe1-bbb5-5bb39bba66c1', metadata={'Header 1': 'Serene Solace Overview', 'Header 2': '**Data Policy**'}, page_content='We do not store the chat data beyond a chat session.'), Document(id='621c573b-0da1-4cff-bd26-638d79ea41ef', metadata={'Header 1': 'Serene Solace Overview', 'Header 2': '**Key Services**'}, page_content='- **Immediate emotional support** for those feeling stressed, lonely, anxious, or just needing someone to talk to.\n- **Access to professional therapists** for more complex mental health challenges.\n- **Affordable mental health support** to ensure that financial constraints do not prevent you from getting help.  \n---')]